<font color='darkred'> Unless otherwise noted, **this notebook will not be reviewed or autograded.**</font> You are welcome to use it for scratchwork, but **only the files listed in the exercises will be checked.**

---

# Exercises

For these exercises, you'll be creating a [Python class](https://www.hackerearth.com/practice/python/object-oriented-programming/classes-and-objects-i/tutorial/) in the *apputil.py* file.

A few tips:

- Some of these exercises will require some "JSON digging".
- <font color='lightblue'>Your functions should still adhere to [best practices](https://codesignal.com/learn/courses/clean-code-basics-with-python/lessons/clean-function-design-in-python), so **your class should probably contain more functions than only the ones dictated below**.</font>


## Exercise 1

Create a Python class named `Genius` such that the following code initializes the object, and "saves" the access token as an attribute of the object. You'll need to use this attribute for Exercises 2 and 3.

```python
from apputil import Genius

genius = Genius(access_token="access_token")
```

In [9]:
import os
import requests
import pandas as pd
from typing import Optional, List, Dict
from dotenv import load_dotenv

# Load .env if available
load_dotenv("week-6.env")


class Genius:
    """
    Minimal Genius API wrapper for exercises and testing.
    """

    def __init__(self, access_token: Optional[str] = None):
        # Prefer explicit token, fallback to environment
        if access_token is None:
            access_token = os.getenv("ACCESS_TOKEN")
        self.access_token = access_token

    def _get(self, url: str, params: Dict = None) -> Dict:
        """Internal helper to make GET requests with Authorization header."""
        headers = {}
        if self.access_token:
            headers["Authorization"] = f"Bearer {self.access_token}"

        response = requests.get(url, params=params, headers=headers, timeout=10)
        response.raise_for_status()  # Let tests catch HTTP errors
        return response.json()

    def get_artist(self, search_term: str) -> Dict:
        """
        Search Genius for an artist by name.
        Returns parsed JSON from the /artists/{id} endpoint or {} if not found.
        """
        if not isinstance(search_term, str) or not search_term.strip():
            raise ValueError("search_term must be a non-empty string")

        search_url = "https://api.genius.com/search"
        search_json = self._get(search_url, params={"q": search_term})

        hits = search_json.get("response", {}).get("hits", [])
        if not hits:
            return {}

        artist_id = hits[0].get("result", {}).get("primary_artist", {}).get("id")
        if not artist_id:
            return {}

        artist_url = f"https://api.genius.com/artists/{artist_id}"
        return self._get(artist_url)

    def get_artists(self, search_terms: List[str]) -> pd.DataFrame:
        """
        Accepts a list of search terms and returns a DataFrame with:
        ['search_term', 'artist_name', 'artist_id', 'follower_count']
        """
        if not isinstance(search_terms, (list, tuple)):
            raise ValueError("search_terms must be a list or tuple")

        records = []
        for term in search_terms:
            artist_data = {
                "search_term": term,
                "artist_name": None,
                "artist_id": None,
                "follower_count": None
            }

            try:
                json_data = self.get_artist(term)
            except requests.HTTPError:
                records.append(artist_data)
                continue

            artist_obj = json_data.get("response", {}).get("artist")
            if artist_obj:
                artist_data["artist_name"] = artist_obj.get("name")
                artist_data["artist_id"] = artist_obj.get("id")
                artist_data["follower_count"] = artist_obj.get("followers_count")

            records.append(artist_data)

        return pd.DataFrame(records)


# Only runs if file executed directly, not when imported by tests
if __name__ == "__main__":
    g = Genius()
    print("Token loaded:", bool(g.access_token))
    # Manual check (commented to avoid accidental live calls)
    # print(g.get_artist("Missy Elliott"))
    # print(g.get_artists(["Drake", "Taylor Swift"]))


Token loaded: True


In [5]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd

# Load environment variables
load_dotenv("week-6.env")

class Genius:
    def __init__(self):
        self.access_token = os.getenv("ACCESS_TOKEN")
        if not self.access_token:
            raise ValueError("ACCESS_TOKEN not found. Make sure it is set in week-6.env")

    def get_artist(self, search_term):
        """Returns full artist JSON object for a given search term."""
        search_url = f"https://api.genius.com/search?q={search_term}&access_token={self.access_token}"
        response = requests.get(search_url)
        json_data = response.json()

        artist_id = json_data['response']['hits'][0]['result']['primary_artist']['id']
        artist_url = f"https://api.genius.com/artists/{artist_id}?access_token={self.access_token}"
        artist_response = requests.get(artist_url)
        return artist_response.json()

    def get_artists(self, search_terms):
        """Returns a Pandas DataFrame with selected artist info."""
        records = []
        for term in search_terms:
            data = self.get_artist(term)['response']['artist']
            records.append({
                "search_term": term,
                "artist_name": data.get("name"),
                "artist_id": data.get("id"),
                "follower_count": data.get("followers_count")
            })
        return pd.DataFrame(records)


# Run only when file is executed directly (not imported)
if __name__ == "__main__":
    genius = Genius()
    search_terms = ["Shaboozey", "Usher", "Taylor Swift", "Drake"]
    df = genius.get_artists(search_terms)
    print(df)

    search_term   artist_name  artist_id  follower_count
0     Shaboozey     Shaboozey      69508              30
1         Usher         USHER        132             578
2  Taylor Swift  Taylor Swift       1177           10616
3         Drake         Drake        130           17881


In [ ]:
# Exercise 1 

# Create a python class named 'Genius' such that the following code initializes the object, and "saves" the access token as an attribute of the object. You'll need to use this attribute for Exercise 2 and 3.
"""
    Download the week-6.env file 
    and add to the .gitignore;
    Then create python class 'Genius' and connect 
    to the access token in the .env file
"""
from dotenv import load_dotenv
import os
load_dotenv('week-6.env')

class Genius:
    def __init__(self):
        self.access_token = os.getenv('ACCESS_TOKEN')
"""
    Should print the access token
"""
genius = Genius()
print(f"ACCESS_TOKEN: {genius.access_token}")


ACCESS_TOKEN: vw58dWIGES4DS1hb7pYPdbSyEIR_klztMKLcOgK3HtH2qFCn9EESV5p7xs96IiQc


## Exercise 2

Reference the `json_data` in the lab. Notice, when we search for an artist name (e.g., "Missy Elliot") in Genius, the result is a list of *songs* attributed to that artist. Suppose we want to capture information about the artist themselves.

Create a method for our `Genius` class called `.get_artist(search_term)` which does the following:

1. Extract the (most likely, "Primary") Artist ID from the first "hit" of the `search_term`.
2. Use the [API path](https://docs.genius.com/#artists-h2) for this Artist ID to pull information about the artist.
3. **Return** the dictionary containing the resulting JSON object.

For example, the following code should return a dictionary of artist information:

```python
genius.get_artist("Radiohead")
```

In [ ]:
# Exercise 2

"""
    Define genius class and retrieve access token;
    Search for artist and get the first hit's artist ID
"""
import requests
import pandas as pd
import os

class Genius:
    def __init__(self):
        self.access_token = os.getenv('ACCESS_TOKEN')
    def get_artist(self, search_term):
        search_url = f"http://api.genius.com/search?q={search_term}&access_token={self.access_token}"
        response = requests.get(search_url)
        json_data = response.json()
      
        """
            Extract the artist ID from the first hit
        """
        artist_id = json_data['response']['hits'][0]['result']['primary_artist']['id']
        """
            Use the artist ID to get the artist information"""
        artist_url = f"http://api.genius.com/artists/{artist_id}?access_token={self.access_token}"
        artist_response = requests.get(artist_url)
        artist_data = artist_response.json()
        """
            Return the resulting JSON object
        """
        return artist_data
genius = Genius()
artist_info = genius.get_artist("Missy Elliott")
print(artist_info) 



{'meta': {'status': 200}, 'response': {'artist': {'alternate_names': ["Missy 'Misdemeanor' Elliott", 'Missy', 'Melissa A. Elliott', 'Melissa Arnette Elliott', 'Melissa Elliott'], 'api_path': '/artists/1529', 'description': {'dom': {'tag': 'root', 'children': [{'tag': 'p', 'children': ['Missy Elliott, also known as Missy “Misdemeanor” Elliott, is a Grammy Award-winning rapper, singer, songwriter, producer and actress. She has released several platinum albums while writing and producing hits for the likes of ', {'tag': 'a', 'attributes': {'href': 'https://genius.com/artists/Aaliyah', 'rel': 'noopener'}, 'data': {'api_path': '/artists/1557'}, 'children': ['Aaliyah']}, ', ', {'tag': 'a', 'attributes': {'href': 'https://genius.com/artists/702', 'rel': 'noopener'}, 'data': {'api_path': '/artists/8468'}, 'children': ['702']}, ', ', {'tag': 'a', 'attributes': {'href': 'https://genius.com/artists/Swv', 'rel': 'noopener'}, 'data': {'api_path': '/artists/3258'}, 'children': ['SWV']}, ', ', {'tag'

## Exercise 3

Use the result from Exercise 2 to create another method for our `Genius` class called `.get_artists(search_terms)` (plural) which takes in a *list* of search terms, and returns a DataFrame containing a row for each search term, and the following columns:

- `search_term`: the raw search term from `search_terms`
- `artist_name`: the (most likely) artist name for the search term
- `artist_id`: the Genius Artist ID for that artist, based on the API call
- `followers_count`: the number of followers for that artist (if available)

For example, the following should return a DataFrame with 4 rows:

```python
genius.get_artists(['Rihanna', 'Tycho', 'Seal', 'U2'])
```

In [ ]:
# Exercise 3

#    Use the result from Exercise 2 to create another method for our 'Genius' class called '.get_artists(search_terms)' (plural) which takes in a list of search terms, and returns a DataFrame containing a row for each search term, and the following columns:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv('week-6.env')

class Genius:
    def __init__(self):
        self.access_token = os.getenv('ACCESS_TOKEN')
    """
        New method to get artist information for a list of search terms
        Getting artist information for various search terms
    """
    def get_artist(self, search_term):
        search_url = f"http://api.genius.com/search?q={search_term}&access_token={self.access_token}"
        response = requests.get(search_url)
        json_data = response.json()
        artist_id = json_data['response']['hits'][0]['result']['primary_artist']['id']
        artist_url = f"http://api.genius.com/artists/{artist_id}?access_token={self.access_token}"
        artist_response = requests.get(artist_url)
        artist_data = artist_response.json()
        return artist_data
    """
        Defining columns and creating DataFrame
        Returns DataFrame with artist details
    """
    def get_artists(self, search_terms):
        records = []
        for term in search_terms:
            artist_info = self.get_artist(term)
            artist_data = artist_info['response']['artist']
            record = {
                'search_term': term,
                'artist_name': artist_data.get('name'),
                'artist_id': artist_data.get('id'),
                'follower_count': artist_data.get('followers_count')
            }
            records.append(record)
        df = pd.DataFrame(records)
        return df
genius = Genius()
search_terms = ["Shaboozey", "Usher", "Taylor Swift", "Drake"]
df = genius.get_artists(search_terms)
print(df)

    search_term   artist_name  artist_id  follower_count
0     Shaboozey     Shaboozey      69508              29
1         Usher         USHER        132             576
2  Taylor Swift  Taylor Swift       1177           10582
3         Drake         Drake        130           17841


## Bonus Exercise (optional)

1. Gather a list of 100+ various musical artists, and save this list in a TXT file.
2. Write a Python *script* that saves the result of `.get_artists` for these artists in a CSV file.
3. If you have time, adjust this script to use multiprocessing.